In [1]:
from mxnet.gluon import nn

net = nn.Sequential()
drop_prob1 = 0.2
drop_prob2 = 0.5

with net.name_scope():
    net.add(nn.Flatten())
    # 1st dense layer
    net.add(nn.Dense(256, activation = 'relu'))
    # add dropout
    net.add(nn.Dropout(drop_prob1))
    # 2nd dense layer
    net.add(nn.Dense(256, activation = 'relu'))
    # add dropout
    net.add(nn.Dropout(drop_prob2))
    net.add(nn.Dense(10))
    

net.initialize()

/home/dyjng/anaconda3/lib/python3.6/site-packages/urllib3/contrib/pyopenssl.py:46: DeprecationWarning: OpenSSL.rand is deprecated - you should use os.urandom instead
  import OpenSSL.SSL


In [2]:
import sys
sys.path.append('..')
import utils
from mxnet import nd
from mxnet import autograd as ag
from mxnet import gluon

batch_size = 256
train_data, test_data = utils.load_data_fashion_mnist(batch_size)

softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.5})

for epoch in range(5):
    train_loss = 0
    train_acc = 0
    for data, label in train_data:
        with ag.record():
            output = net(data)
            loss = softmax_cross_entropy(output, label)
        loss.backward()
        trainer.step(batch_size)
        
        # there train_loss and train_acc represent sum every batch value one by one
        train_loss += nd.mean(loss).asscalar()
        train_acc += utils.accuracy(output, label)
    
    test_acc = utils.evaluate_accuracy(test_data, net)
    print('Epoch %d, Loss: %f, Train acc %f, Test acc %f' % (
        epoch, train_loss / len(train_data), train_acc / len(train_data), test_acc))

Epoch 0, Loss: 0.815891, Train acc 0.698150, Test acc 0.821314
Epoch 1, Loss: 0.511733, Train acc 0.811899, Test acc 0.847155
Epoch 2, Loss: 0.447136, Train acc 0.836922, Test acc 0.861378
Epoch 3, Loss: 0.421636, Train acc 0.844768, Test acc 0.859375
Epoch 4, Loss: 0.395901, Train acc 0.854033, Test acc 0.867688
